In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

from src.inference import fetch_next_hour_predictions

In [3]:
predictions = fetch_next_hour_predictions()

2025-05-05 11:28:42,469 INFO: Initializing external client
2025-05-05 11:28:42,469 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 11:28:43,672 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 
Current EST time: 2025-05-05 07:28:42.468888
Next hour: 2025-05-05 08:00:00
Found 3 records


In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
station_dict = {
    np.float32(5905.140137): {"name": "Broadway & E 14 St",
           "longitude": -73.99074142,
           "latitude": 40.73454567
           },
    np.float32(6140.049805): {"name": "W 21 St & 6 Ave",
           "longitude": -73.99415556,
           "latitude": 40.74173969},
    np.float32(6822.089844): {"name": "1 Ave & E 68 St",
           "longitude": -73.958115339,
           "latitude": 40.765112281}
}

station_df = pd.DataFrame.from_dict(station_dict, orient='index').reset_index()
station_df.rename(columns={'index': 'start_station_id'}, inplace=True)
merged = pd.merge(predictions, station_df, on='start_station_id', how='left')
merged

,start_station_id,predicted_demand,start_hour,name,longitude,latitude
0,6140.049805,7.0,2025-05-05 08:00:00,W 21 St & 6 Ave,-73.994156,40.741740
1,5905.140137,12.0,2025-05-05 08:00:00,Broadway & E 14 St,-73.990741,40.734546
2,6822.089844,8.0,2025-05-05 08:00:00,1 Ave & E 68 St,-73.958115,40.765112


In [7]:
station_df

,start_station_id,name,longitude,latitude
0,5905.140137,Broadway & E 14 St,-73.990741,40.734546
1,6140.049805,W 21 St & 6 Ave,-73.994156,40.741740
2,6822.089844,1 Ave & E 68 St,-73.958115,40.765112


In [8]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   start_station_id  3 non-null      float32
 1   name              3 non-null      object 
 2   longitude         3 non-null      float64
 3   latitude          3 non-null      float64
dtypes: float32(1), float64(2), object(1)
memory usage: 216.0+ bytes


In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_station_id  3 non-null      float32       
 1   predicted_demand  3 non-null      float64       
 2   start_hour        3 non-null      datetime64[us]
 3   name              3 non-null      object        
 4   longitude         3 non-null      float64       
 5   latitude          3 non-null      float64       
dtypes: datetime64[us](1), float32(1), float64(3), object(1)
memory usage: 264.0+ bytes


In [10]:
merged

,start_station_id,predicted_demand,start_hour,name,longitude,latitude
0,6140.049805,7.0,2025-05-05 08:00:00,W 21 St & 6 Ave,-73.994156,40.741740
1,5905.140137,12.0,2025-05-05 08:00:00,Broadway & E 14 St,-73.990741,40.734546
2,6822.089844,8.0,2025-05-05 08:00:00,1 Ave & E 68 St,-73.958115,40.765112


In [11]:
merged["start_station_id"].round(0)

0    6140.0
1    5905.0
2    6822.0
Name: start_station_id, dtype: float32

In [12]:
import folium

map_center = [40.76, -73.98]
map_obj = folium.Map(location=map_center, zoom_start=12)

for _, row in merged.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=8,
        popup=f"{row['name']}: {row['predicted_demand']:.0f} rides",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
    ).add_to(map_obj)

display(map_obj)

In [51]:
from src.inference import load_batch_of_features_from_store, fetch_hourly_rides
current_date = pd.Timestamp.now(tz="US/Eastern")
features = fetch_hourly_rides(672)
predictions = fetch_next_hour_predictions()

2025-05-05 12:03:50,273 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 12:03:50,280 INFO: Initializing external client
2025-05-05 12:03:50,281 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 12:03:51,242 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 
2025-05-05 12:03:54,182 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 12:03:54,188 INFO: Initializing external client
2025-05-05 12:03:54,188 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 12:03:54,910 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 
Current EST time: 2025-05-05 08:03:54.182266
Next hour: 2025-05-05 09:00:00
Found 3 reco

In [14]:
features["start_hour"].max()

Timestamp('2025-05-04 16:00:00')

In [15]:
predictions

,start_station_id,predicted_demand,start_hour
51,6140.049805,7.0,2025-05-05 08:00:00
52,5905.140137,12.0,2025-05-05 08:00:00
53,6822.089844,8.0,2025-05-05 08:00:00


In [52]:
features

,start_hour,start_station_id,rides
0,2025-05-02 09:00:00-04:00,6822.089844,17
1,2025-04-24 13:00:00-04:00,5905.140137,21
2,2025-05-02 23:00:00-04:00,6822.089844,4
3,2025-04-23 23:00:00-04:00,5905.140137,5
4,2025-05-03 15:00:00-04:00,5905.140137,47
...,...,...,...
2047,2025-05-05 14:00:00-04:00,6822.089844,29
2048,2025-05-05 14:00:00-04:00,5905.140137,32
2049,2025-05-05 15:00:00-04:00,6822.089844,38
2050,2025-05-05 15:00:00-04:00,5905.140137,26


In [21]:
import pandas as pd
import plotly.express as px
from plotly.graph_objects import Figure
from typing import Optional
from datetime import timedelta

def plot_ts(features: pd.DataFrame, predictions: pd.DataFrame, location_id: float) -> Figure:
    location_features = features[features["start_station_id"] == location_id]
    ts_cols = [col for col in features.columns if col.startswith("rides_t-")]
    ts_values = [location_features[col] for col in ts_cols]

    # Generate timestamps
    start_time = pd.to_datetime(location_features["start_hour"].iloc[0])
    time_series_dates = pd.date_range(
        start=start_time - timedelta(hours=len(ts_cols)),
        end=start_time,
        freq="h",
    )

    # Title
    title = f"Start Hour: {location_features['start_hour']}, Start Station ID: {location_features['start_station_id']}"

    # Base plot
    fig = px.line(
        x=time_series_dates,
        y=ts_values,
        template="plotly_white",
        markers=True,
        title=title,
        labels={"x": "Time", "y": "Ride Counts"},
    )

    return fig

In [58]:
location_features = features[features["start_station_id"] == 5905.140137]
location_features = location_features.sort_values(by=["start_station_id", "start_hour"], ascending=True)
location_features

,start_hour,start_station_id,rides
1009,2025-04-07 04:00:00-04:00,5905.140137,1
752,2025-04-07 05:00:00-04:00,5905.140137,1
933,2025-04-07 06:00:00-04:00,5905.140137,4
637,2025-04-07 07:00:00-04:00,5905.140137,13
1273,2025-04-07 08:00:00-04:00,5905.140137,19
...,...,...,...
2037,2025-05-05 11:00:00-04:00,5905.140137,15
2041,2025-05-05 12:00:00-04:00,5905.140137,27
2044,2025-05-05 13:00:00-04:00,5905.140137,15
2048,2025-05-05 14:00:00-04:00,5905.140137,32


In [35]:
ts_cols = [col for col in features.columns if col.startswith("rides_t-")]
ts_values = [location_features[col] for col in ts_cols]
ts_values = [s.item() for s in ts_values]
ts_values

[34,
 20,
 46,
 31,
 31,
 39,
 43,
 20,
 13,
 10,
 10,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 4,
 13,
 19,
 10,
 10,
 23,
 20,
 28,
 16,
 26,
 27,
 40,
 39,
 21,
 23,
 15,
 11,
 3,
 4,
 1,
 0,
 2,
 3,
 1,
 7,
 15,
 34,
 20,
 16,
 24,
 18,
 27,
 25,
 37,
 30,
 54,
 69,
 43,
 27,
 17,
 15,
 10,
 5,
 0,
 0,
 0,
 0,
 2,
 4,
 11,
 26,
 26,
 22,
 20,
 24,
 30,
 26,
 37,
 26,
 33,
 36,
 39,
 29,
 18,
 20,
 15,
 5,
 1,
 1,
 1,
 0,
 0,
 6,
 10,
 10,
 14,
 16,
 11,
 18,
 30,
 24,
 31,
 25,
 21,
 23,
 16,
 11,
 13,
 6,
 7,
 4,
 2,
 0,
 0,
 1,
 1,
 4,
 9,
 11,
 23,
 17,
 24,
 12,
 7,
 22,
 33,
 30,
 31,
 44,
 28,
 20,
 14,
 12,
 13,
 9,
 2,
 3,
 4,
 2,
 1,
 1,
 2,
 6,
 8,
 22,
 18,
 24,
 26,
 31,
 28,
 19,
 30,
 27,
 18,
 15,
 18,
 9,
 9,
 8,
 3,
 0,
 2,
 3,
 0,
 1,
 3,
 4,
 12,
 22,
 13,
 24,
 24,
 41,
 36,
 46,
 22,
 33,
 27,
 18,
 5,
 1,
 5,
 4,
 2,
 4,
 6,
 2,
 1,
 11,
 17,
 25,
 19,
 18,
 19,
 22,
 24,
 28,
 42,
 32,
 47,
 50,
 41,
 36,
 20,
 8,
 4,
 0,
 1,
 0,
 0,
 1,
 1,
 7,
 15,
 31,
 22,
 13,
 19,


In [28]:
start_time = pd.to_datetime(location_features["start_hour"].iloc[0])
time_series_dates = pd.date_range(
    start=start_time - timedelta(hours=len(ts_cols)),
    end=start_time - timedelta(hours=1),
    freq="h",
)
time_series_dates

DatetimeIndex(['2025-04-06 16:00:00', '2025-04-06 17:00:00',
               '2025-04-06 18:00:00', '2025-04-06 19:00:00',
               '2025-04-06 20:00:00', '2025-04-06 21:00:00',
               '2025-04-06 22:00:00', '2025-04-06 23:00:00',
               '2025-04-07 00:00:00', '2025-04-07 01:00:00',
               ...
               '2025-05-04 06:00:00', '2025-05-04 07:00:00',
               '2025-05-04 08:00:00', '2025-05-04 09:00:00',
               '2025-05-04 10:00:00', '2025-05-04 11:00:00',
               '2025-05-04 12:00:00', '2025-05-04 13:00:00',
               '2025-05-04 14:00:00', '2025-05-04 15:00:00'],
              dtype='datetime64[ns]', length=672, freq='H')

In [78]:
from src.inference import fetch_predictions

preds = fetch_predictions(672)
preds

2025-05-05 12:52:52,433 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 12:52:52,439 INFO: Initializing external client
2025-05-05 12:52:52,440 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 12:52:53,166 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


,start_station_id,predicted_demand,start_hour
0,6822.089844,18.0,2025-05-04 12:00:00-04:00
1,5905.140137,21.0,2025-05-04 12:00:00-04:00
2,6140.049805,21.0,2025-05-04 12:00:00-04:00
3,6822.089844,9.0,2025-05-04 14:00:00-04:00
4,5905.140137,23.0,2025-05-04 14:00:00-04:00
...,...,...,...
64,5905.140137,6.0,2025-05-05 12:00:00-04:00
65,6140.049805,18.0,2025-05-05 12:00:00-04:00
66,6822.089844,16.0,2025-05-05 13:00:00-04:00
67,5905.140137,5.0,2025-05-05 13:00:00-04:00


In [85]:
current_hour_utc = (pd.Timestamp.now(tz="UTC") - timedelta(hours=672)).floor("h")
current_hour_utc
pd.Timestamp.now(tz="UTC")

Timestamp('2025-05-05 16:58:47.776817+0000', tz='UTC')

In [89]:
from src.inference import fetch_hourly_rides

test = fetch_hourly_rides(672).sort_values(["start_station_id", "start_hour"])

2025-05-05 13:06:04,969 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 13:06:04,976 INFO: Initializing external client
2025-05-05 13:06:04,976 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 13:06:05,709 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


In [90]:
test

,start_hour,start_station_id,rides
60795,2025-04-07 13:00:00-04:00,5905.140137,28
61046,2025-04-07 14:00:00-04:00,5905.140137,16
59552,2025-04-07 15:00:00-04:00,5905.140137,26
60724,2025-04-07 16:00:00-04:00,5905.140137,27
60296,2025-04-07 17:00:00-04:00,5905.140137,40
...,...,...,...
61524,2025-05-05 09:00:00-04:00,6822.089844,23
61527,2025-05-05 10:00:00-04:00,6822.089844,14
61532,2025-05-05 11:00:00-04:00,6822.089844,10
61533,2025-05-05 12:00:00-04:00,6822.089844,16


In [87]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2031 entries, 60769 to 61545
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   start_hour        2031 non-null   datetime64[us, Etc/UTC]
 1   start_station_id  2031 non-null   float32                
 2   rides             2031 non-null   int32                  
dtypes: datetime64[us, Etc/UTC](1), float32(1), int32(1)
memory usage: 47.6 KB


In [46]:
preds = preds.sort_values(by=["start_station_id", "start_hour"], ascending=True)

In [47]:
location_preds = preds[preds["start_station_id"] == 5905.140137]

In [43]:
preds["start_hour"].max()

Timestamp('2025-05-05 12:00:00-0400', tz='US/Eastern')

In [44]:
preds_dates = pd.date_range(
    start=preds["start_hour"].min(),
    end=preds["start_hour"].max(),
    freq="h",
)
preds_dates

DatetimeIndex(['2025-05-04 12:00:00-04:00', '2025-05-04 13:00:00-04:00',
               '2025-05-04 14:00:00-04:00', '2025-05-04 15:00:00-04:00',
               '2025-05-04 16:00:00-04:00', '2025-05-04 17:00:00-04:00',
               '2025-05-04 18:00:00-04:00', '2025-05-04 19:00:00-04:00',
               '2025-05-04 20:00:00-04:00', '2025-05-04 21:00:00-04:00',
               '2025-05-04 22:00:00-04:00', '2025-05-04 23:00:00-04:00',
               '2025-05-05 00:00:00-04:00', '2025-05-05 01:00:00-04:00',
               '2025-05-05 02:00:00-04:00', '2025-05-05 03:00:00-04:00',
               '2025-05-05 04:00:00-04:00', '2025-05-05 05:00:00-04:00',
               '2025-05-05 06:00:00-04:00', '2025-05-05 07:00:00-04:00',
               '2025-05-05 08:00:00-04:00', '2025-05-05 09:00:00-04:00',
               '2025-05-05 10:00:00-04:00', '2025-05-05 11:00:00-04:00',
               '2025-05-05 12:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', freq='H')

In [55]:
location_features['start_station_id'].iloc[0]

5905.14

In [61]:
station_dict = {
    int(5905.0): {"name": "Broadway & E 14 St",
           "longitude": -73.99074142,
           "latitude": 40.73454567
           },
    int(6140.0): {"name": "W 21 St & 6 Ave",
           "longitude": -73.99415556,
           "latitude": 40.74173969},
    int(6822.0): {"name": "1 Ave & E 68 St",
           "longitude": -73.958115339,
           "latitude": 40.765112281}
}
station_df = pd.DataFrame.from_dict(station_dict, orient='index').reset_index()
station_df.rename(columns={'index': 'start_station_id'}, inplace=True)

In [76]:
rides = fetch_hourly_rides(672)
preds = fetch_predictions(672)

preds["start_station_id"] = preds["start_station_id"].round(0).astype(int)
preds = pd.merge(preds, station_df, on='start_station_id', how='left')
rides["start_station_id"] = rides["start_station_id"].round(0).astype(int)
rides = pd.merge(rides, station_df, on='start_station_id', how='left')

2025-05-05 12:47:10,761 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 12:47:10,768 INFO: Initializing external client
2025-05-05 12:47:10,769 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 12:47:11,526 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 
2025-05-05 12:47:14,266 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-05 12:47:14,271 INFO: Initializing external client
2025-05-05 12:47:14,271 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-05 12:47:15,007 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


In [65]:
station_options = preds["name"].dropna().unique()
station_options

array(['Broadway & E 14 St', 'W 21 St & 6 Ave', '1 Ave & E 68 St'],
      dtype=object)

In [67]:
rides

,start_hour,start_station_id,rides,name,longitude,latitude
0,2025-05-02 09:00:00-04:00,6822,17,1 Ave & E 68 St,-73.958115,40.765112
1,2025-04-24 13:00:00-04:00,5905,21,Broadway & E 14 St,-73.990741,40.734546
2,2025-05-02 23:00:00-04:00,6822,4,1 Ave & E 68 St,-73.958115,40.765112
3,2025-04-23 23:00:00-04:00,5905,5,Broadway & E 14 St,-73.990741,40.734546
4,2025-05-03 15:00:00-04:00,5905,47,Broadway & E 14 St,-73.990741,40.734546
...,...,...,...,...,...,...
2047,2025-05-05 14:00:00-04:00,6822,29,1 Ave & E 68 St,-73.958115,40.765112
2048,2025-05-05 14:00:00-04:00,5905,32,Broadway & E 14 St,-73.990741,40.734546
2049,2025-05-05 15:00:00-04:00,6822,38,1 Ave & E 68 St,-73.958115,40.765112
2050,2025-05-05 15:00:00-04:00,5905,26,Broadway & E 14 St,-73.990741,40.734546


In [68]:
selected_station = "Broadway & E 14 St"

In [69]:
# Filter data for the selected station
preds_station_data = preds[preds["name"] == selected_station].sort_values("start_hour")
rides_station_data = rides[rides["name"] == selected_station].sort_values("start_hour")

In [71]:
rides_station_data
preds_station_data

,start_station_id,predicted_demand,start_hour,name,longitude,latitude
1,5905,21.0,2025-05-04 12:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
4,5905,23.0,2025-05-04 14:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
7,5905,18.0,2025-05-04 16:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
9,5905,12.0,2025-05-04 17:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
12,5905,3.0,2025-05-04 18:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
16,5905,18.0,2025-05-04 19:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
20,5905,20.0,2025-05-04 20:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
23,5905,5.0,2025-05-04 21:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
24,5905,28.0,2025-05-04 23:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546
27,5905,24.0,2025-05-05 00:00:00-04:00,Broadway & E 14 St,-73.990741,40.734546


In [74]:
import plotly.graph_objects as go

fig = go.Figure()
# Base plot
fig.add_trace(go.Scatter(
    x=rides_station_data["start_hour"],
    y=rides_station_data["rides"],
    mode="lines+markers",
    name="Actual Rides"
))

fig.add_trace(go.Scatter(
    x=preds_station_data["start_hour"],
    y=preds_station_data["predicted_demand"],
    mode="lines+markers",
    name="Predicted Demand",
    line=dict(dash='dash', color="orange")
))

fig.update_layout(
    title=f"Ride Counts and Predicted Demand for Station @ {selected_station}",
    xaxis_title="Time",
    yaxis_title="Rides",
    template="plotly_white"
)

fig.show()

In [59]:
import plotly.graph_objects as go

fig = go.Figure()
# Base plot
fig.add_trace(go.Scatter(
    x=location_features["start_hour"],
    y=location_features["rides"],
    mode="lines+markers",
    name="Actual Rides"
))

fig.add_trace(go.Scatter(
    x=location_preds["start_hour"],
    y=location_preds["predicted_demand"],
    mode="lines+markers",
    name="Predicted Demand",
    line=dict(dash='dash', color="orange")
))

fig.update_layout(
    title=f"Ride Counts and Predicted Demand for Station {location_features['start_station_id'].iloc[0]}",
    xaxis_title="Time",
    yaxis_title="Rides",
    template="plotly_white"
)

fig.show()

In [22]:
plot_ts(features, predictions, location_id=5905.140137)

ValueError: All arguments should have the same length. The length of argument `wide_variable_0` is 1, whereas the length of previously-processed arguments ['x'] is 673

In [ ]:
from src.plot_utils import plot_prediction2


AttributeError: 'NoneType' object has no attribute 'columns'